### Advance PDF parsing with easyparser

In [ ]:
from pathlib import Path
pdf_path = str(Path.cwd().parent / "tests" / "assets" / "long.pdf")

In [ ]:
from easyparser.base import Chunk, ChunkGroup
from easyparser.controller import Controller
from easyparser.parser.pdf import FastPDF
from easyparser.split.toc_builder import TOCExtractor

# load the PDF file with easyparser
ctrl = Controller()
root = ctrl.as_root_chunk(pdf_path)
chunks: ChunkGroup = FastPDF.run(
    root, 
    use_layout_parser=True,
)
# print the chunks graph
chunks[0].print_graph()

 Chunk(id=a67e1, ctype=root, origin=/Users/taprosoft/Shared/debug/chunking/tests/assets/long.pdf)
     Chunk(id=177f1, ctype=figure, mimetype=image/png)
     Chunk(id=d566e, ctype=header, content=DeepSeek-V3 Technical Report)
     Chunk(id=d9fd5, ctype=para, content=DeepSeek-AI)
     Chunk(id=52c81, ctype=para, content=research@deepseek.com)
     Chunk(id=9ce24, ctype=header, content=Abstract)
     Chunk(id=41420, ctype=para, content=We present DeepSeek-V3, a strong Mixture-of-Expert... (142 more words))
     Chunk(id=677f6, ctype=figure, mimetype=image/png)
     Chunk(id=ae7ca, ctype=para, content=Figure 1 | Benchmark performance of DeepSeek-V3 and its counterparts.)
     Chunk(id=88d37, ctype=para, content=arXiv:2412.19437v1 [cs.CL] 27 Dec 2024)
     Chunk(id=e7794, ctype=header, content=Contents)
     Chunk(id=882eb, ctype=para, content=**1 Introduction 4**)
     Chunk(id=60af6, ctype=table, mimetype=image/png)
     Chunk(id=87679, ctype=para, content=4.5.2 Ablation Studies for the 

In [ ]:
# specify the LLM model name to use for TOC extraction
# os.environ["GEMINI_API_KEY"] = "your_gemini_api_key"
model = "gemini-2.5-flash-preview-04-17"

# if the PDF metadata has TOC information, it will be pulled directly
# if not, the TOC will be extracted using the LLM based on the headers
chunks = TOCExtractor.run(chunks, model=model, use_llm=True)
# print the chunks graph
print(chunks[0].render())

# Introduction
# Architecture
## Basic Architecture
### Multi-Head Latent Attention
### DeepSeekMoE with Auxiliary-Loss-Free Load Balancing
## Multi-Token Prediction
# Infrastructures
## Compute Clusters
## Training Framework
### DualPipe and Computation-Communication Overlap
### Efficient Implementation of Cross-Node All-to-All Communication
### Extremely Memory Saving with Minimal Overhead
## FP8 Training
### Mixed Precision Framework
### Improved Precision from Quantization and Multiplication
### Low-Precision Storage and Communication
## Inference and Deployment
### Prefilling
### Decoding
## Suggestions on Hardware Design
### Communication Hardware
### Compute Hardware
# Pre-Training
## Data Construction
## Hyper-Parameters
## Long Context Extension
## Evaluations
### Evaluation Benchmarks
### Evaluation Results
## Discussion
### Ablation Studies for Multi-Token Prediction
### Ablation Studies for the Auxiliary-Loss-Free Balancing Strategy
### Batch-Wise Load Balance VS. Sequence-